## Model from hugging face: https://huggingface.co/google/pegasus-cnn_dailymail

## Dataset from hugging face: https://huggingface.co/datasets/samsum

In [1]:
!nvidia-smi

Tue Jan 23 22:25:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

```
Below command installs a set of essential libraries for working with natural language processing tasks, including:

Training and using pre-trained language models (Transformers)
Handling various text datasets (Datasets)
Evaluating model performance for machine translation (SacreBLEU) and text summarization (ROUGE)
Working with compressed datasets (py7zr)
```

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.9 M

```
Upgrading all the installed packages
```

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
Found existing installation: accelerate 0.26.1
Uninstalling accelerate-0.26.1:
  Successfully uninstalled accelerate-0.26.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.1 MB/s eta 0:00:00
  Using cached accelerate-0.26.1-py3-none-any.whl (270 kB)


```
Importing all the required library
```

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

```
Using GPU if available other wise CPU
```

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

```
1. Now doing word embedding through AutoTokenizer to convert text to no. to provide to model for further prediction.
2. Here we are using hugging pretrain model for this word embedding

### Initializing variable for model which we will use for summarization
```

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

```
Now lets load the model and as we are using pytoch so we have added to(device) at the end for loading my model for GPU, since we are using GPU
```

In [7]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
dataset_samsum = load_dataset("samsum")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

### Lets see the the dataset which loaded with training, validation and testing

In [9]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [10]:
dataset_samsum['train']['dialogue'][0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [12]:
dataset_samsum['train'][0]['summary']

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [13]:
dataset_samsum['train'][1]

{'id': '13728867',
 'dialogue': 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
 'summary': 'Olivia and Olivier are voting for liberals in this election. '}

### Here we are Checking how many different dataset is present for training, testing and validation

### Also checking their Dialogue and SUmmary

In [14]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [15]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )   # Coverting X data into no. by embedding

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )  # Coverting target data into no. by embedding

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

### Giving my data to the above function to convert it to no. by embedding

In [16]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [17]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

### 3 new feature has been added in the data after doing word embedding

In [18]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [22]:
dataset_samsum['train']

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [20]:
dataset_samsum_pt['train'][1]['attention_mask']

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [21]:
dataset_samsum_pt['train'][1]['labels']

[18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]

### If dataset is huge I need to give the data in batches, that is why we are using data_collector library here

In [23]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

### Now we are defining training arumensts of the model with necessary which is required.

In [24]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,  #Here giving which directory we want, no. of epochs and rest all values default
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

## Defining trainer variable for dataset which we want train

```
In below code:
train_dataset=dataset_samsum_pt["test"]  --> here giving test data from embedded dataset for less time in training but we can give training data for training the completely```

In [27]:
trainer = Trainer(model=model_pegasus, args=trainer_args,  # Here giving model name, training arguments
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator, # providing tokenizer variable and data_collector variable
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

# trainer = Trainer(model=model_pegasus, args=trainer_args,  # Here giving model name, training arguments
#                   tokenizer=tokenizer, data_collator=seq2seq_data_collator, # providing tokenizer variable and data_collector variable
#                   train_dataset=dataset_samsum_pt["train"],
#                   eval_dataset=dataset_samsum_pt["validation"])

In [29]:
# As we are using pytorch so we need to train the model like this

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=2.458646437701057, metrics={'train_runtime': 146.6152, 'train_samples_per_second': 5.586, 'train_steps_per_second': 0.348, 'total_flos': 319254778601472.0, 'train_loss': 2.458646437701057, 'epoch': 1.0})

## Defining functions for evalutating the model

In [30]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


### Defining the metrics name which we find for evaluating the model

In [31]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

<ipython-input-31-5a43aadd1b0e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


### Now finding the model metrics score for evaluation for rough score

```
Here we need to provide the training the data for all training data but we are providing test data to check the metrics score in less time.```

In [32]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 5/5 [00:15<00:00,  3.04s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.02397,0.0,0.024342,0.023736


## Above rouge score should be high but as we are using testing data, hence it is coming less

In [33]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [34]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [35]:
#Loading tonkenizer which we just saved for prediction further

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

##### Here we are doing prediction from the model. Also providing sample text (dialogue) reference text ( as summary) and then Model will predict its owen summaary.

#### For getting exact summary we increase the epochs size previously to train the model more accurately

In [36]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}  # This all parameter will decide how long summary we wants to see



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda can't find Betty's number .<n>Larry called Hannah's friend Betty last time they were at the park together .<n>Hannah: I'd rather you texted him .


### Lets check for new data

In [37]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}  # This all parameter will decide how long summary we wants to see



sample_text = """Mikayla found her seat on the plane and wiped the tears from her eyes. The sound of her mother’s voice echoed in her ears. “You have to start over, Honey. You and your team are the only hope we have left.” The weight of it all was too much. Especially for an eighteen-year-old. She held her father’s briefcase close to her chest as she took her seat next to the window.

The chaos continued outside beyond the airport walls. Smoke filled the air, and it seemed like the whole world was on fire. People, driven by their aggression, were rushing onto the tarmac. They’d reach the plane soon if the pilot didn’t take off, and Mikayla prayed it wouldn’t be too late.

“Why can’t you come with me?” Mikayla had pleaded with her mom.

“You know that’s not possible,” she had replied. “This is your journey to take.” A look of sadness had filled her eyes. “I can already feel the change coming.”

Mikayla felt so alone. Her father succumbed to the disease only days after it was released. Everything had changed so quickly. All because of a vaccine gone wrong.

Mikayla watched as more people her age and younger filed into the airplane. She wondered which of the older boys she would soon align herself with. She didn’t know any of them. She missed the world in which you got to choose who you fell in love with. Now she wondered if genuine romantic love could even exist in this world. She had always wanted to be a mom someday, but not like this."""

reference = """ The chaos continued outside beyond the airport walls. Smoke filled the air, and it seemed like the whole world was on fire. People, driven by their aggression, were rushing onto the tarmac. They’d reach the plane soon if the pilot didn’t take off, and Mikayla prayed it wouldn’t be too late."""

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:
Mikayla found her seat on the plane and wiped the tears from her eyes. The sound of her mother’s voice echoed in her ears. “You have to start over, Honey. You and your team are the only hope we have left.” The weight of it all was too much. Especially for an eighteen-year-old. She held her father’s briefcase close to her chest as she took her seat next to the window.

The chaos continued outside beyond the airport walls. Smoke filled the air, and it seemed like the whole world was on fire. People, driven by their aggression, were rushing onto the tarmac. They’d reach the plane soon if the pilot didn’t take off, and Mikayla prayed it wouldn’t be too late.

“Why can’t you come with me?” Mikayla had pleaded with her mom.

“You know that’s not possible,” she had replied. “This is your journey to take.” A look of sadness had filled her eyes. “I can already feel the change coming.”

Mikayla felt so alone. Her father succumbed to the disease only days after it was released. Everythi

In [38]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}  # This all parameter will decide how long summary we wants to see



sample_text = """
A: Devi, have you thought about where would you like to go to dinner on Friday for your birthday?

B: I am not sure. I don't know that many restaurants around here.

A: You know, we could look online at the local Internet sites.

B: Good. Let's take a look!

A: What kind of food would you like for your birthday?

B: I enjoy Thai or Japanese the best.

A: This one, Shogun, looks good.

B: Oh yes, I've heard of that one. Everyone I've spoken with says that it is great!

A: Would you like to go there then?

B: I think that that would be a really good choice! Let's call and make a reservation.
"""

reference = """Devi is unsure about where to go for dinner on Friday for her birthday.
She suggests looking online for local restaurants and choosing Thai or Japanese food.
Devi agrees and suggests Shogun, a popular restaurant, as a good choice. They plan to make a reservation."""

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:

A: Devi, have you thought about where would you like to go to dinner on Friday for your birthday?

B: I am not sure. I don't know that many restaurants around here.

A: You know, we could look online at the local Internet sites.

B: Good. Let's take a look!

A: What kind of food would you like for your birthday?

B: I enjoy Thai or Japanese the best.

A: This one, Shogun, looks good.

B: Oh yes, I've heard of that one. Everyone I've spoken with says that it is great!

A: Would you like to go there then?

B: I think that that would be a really good choice! Let's call and make a reservation.


Reference Summary:
Devi is unsure about where to go for dinner on Friday for her birthday. 
She suggests looking online for local restaurants and choosing Thai or Japanese food. 
Devi agrees and suggests Shogun, a popular restaurant, as a good choice. They plan to make a reservation.

Model Summary:
B: I enjoy Thai or Japanese the best. B: I think that that would be a really good choice!